In [1]:
# COMENTARIO: EN CASO DE HABER ELIMINADO VARIABLES EN EL PRE-PROCESAMIENTO, SE DEBEN QUITAR EN LAS COLUMNAS
# CALCULADAS.

## A. Configuración general.

In [59]:
#1. Librerías.
%run "../librerias.ipynb"

In [60]:
#2. Constantes.
%run "../constantes.ipynb"

dataset_input = dataset_file_preprocesado_all_3
dataset_output = dataset_file_fe_all_3
ventana = 3

In [61]:
#3. Funciones.
%run "../funciones.ipynb"

In [62]:
#4. Configuración de jupysql.
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [63]:
#5. Conexión a DuckDB.
%sql duckdb:///:memory:

In [64]:
#6. Lectura de datos.

In [65]:
%%sql
--ROLLBACK;
create or replace table competencia_02 as
select
    *
from read_csv_auto("{{dataset_input}}")

,Success


In [66]:
#7. Funciones útiles.

In [67]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

,Success


In [68]:
%%sql
CREATE OR REPLACE MACRO division_segura(a, b) AS 
    CASE 
        WHEN ifnull(b, 0) = 0 THEN NULL 
        ELSE ifnull(a, 0) / ifnull(b, 1) 
    END;

,Success


## B. Creación de variables intramensuales.

In [69]:
#i. Normalizo ctrx_quarter según la antigüedad del cliente.

In [70]:
%%sql
create or replace table competencia_02_norm as
select *,
    CASE 
        WHEN cliente_antiguedad = 1 THEN ctrx_quarter * 5
        WHEN cliente_antiguedad = 2 THEN ctrx_quarter * 2
        WHEN cliente_antiguedad = 3 THEN ctrx_quarter * 1.2
        ELSE ctrx_quarter
    END AS ctrx_quarter_normalizado,
from competencia_02;

,Success


In [71]:
#i. Sumo Métricas de VISA + MASTERCARD para tener un comportamiento total.

In [72]:
%%sql
create or replace table competencia_02_sumas as
select *,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total
    , suma_sin_null(Master_mfinanciacion_limite,Visa_mfinanciacion_limite) as tc_financiacionlimite_total
    , suma_sin_null(Master_msaldopesos,Visa_msaldopesos) as tc_saldopesos_total
    , suma_sin_null(Master_msaldodolares,Visa_msaldodolares) as tc_saldodolares_total
    , suma_sin_null(Master_mconsumospesos,Visa_mconsumospesos) as tc_consumopesos_total
    , suma_sin_null(Master_mlimitecompra,Visa_mlimitecompra) as tc_limitecompra_total
    , suma_sin_null(Master_madelantopesos,Visa_madelantopesos) as tc_adelantopesos_total
    , suma_sin_null(Master_madelantodolares,Visa_madelantodolares) as tc_adelantodolares_total
    , suma_sin_null(tc_adelantopesos_total,tc_adelantodolares_total) as tc_adelanto_total
    , suma_sin_null(Master_mpagado,Visa_mpagado) as tc_pagado_total
    , suma_sin_null(Master_mpagospesos,Visa_mpagospesos) as tc_pagadopesos_total
    , suma_sin_null(Master_mpagosdolares,Visa_mpagosdolares) as tc_pagadodolares_total
    , suma_sin_null(Master_msaldototal,Visa_msaldototal) as tc_saldototal_total
    , suma_sin_null(Master_mconsumototal,Visa_mconsumototal) as tc_consumototal_total
    , suma_sin_null(Master_cconsumos,Visa_cconsumos) as tc_cconsumos_total
    , suma_sin_null(Master_delinquency,Visa_delinquency) as tc_morosidad_total
from competencia_02_norm;

,Success


In [73]:
%%sql
create or replace table competencia_02_sumas as
select
    *
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
    , greatest(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_mayor
    , least(Master_fechaalta, Visa_fechaalta) as tc_fechalta_menor
    , greatest(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_mayor
    , least(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_menor
    , greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_mayor
    , least(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_menor
from competencia_02_sumas;

,Success


In [74]:
#ii. Sumo Métricas de PESOS + DOLARES y PAYROLLS para tener un comportamiento total (mas allá de las incluidas previamente).

In [75]:
%%sql
create or replace table competencia_02_sumas as
select *
    , suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as m_plazofijo_total
    , suma_sin_null(minversion1_dolares, minversion1_pesos) as m_inversion1_total
    , suma_sin_null(mpayroll, mpayroll2) as m_payroll_total
    , suma_sin_null(cpayroll_trx, cpayroll2_trx) as c_payroll_total
    , suma_sin_null(suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), cseguro_vivienda), cseguro_accidentes_personales) as c_seguros_total
from competencia_02_sumas

,Success


In [76]:
#iii. Calculo Métricas de posición ante antigüedad, fechas de altas, etc.

In [77]:
%%sql
create or replace table competencia_02_sumas as
select
    *
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6 # Corto la poblacion en 10, y le asigno una a cada cliente, según su posición de antigüedad.
from competencia_02_sumas
order by numero_de_cliente, cliente_antiguedad

,Success


In [78]:
%%sql
create or replace table competencia_02_sumas as
select
    *
    ,ntile(10) over (partition by foto_mes order by tc_fechaalta_mayor) as antiguedad_tarjetas
from competencia_02_sumas
order by numero_de_cliente, tc_fechaalta_mayor

,Success


In [79]:
#iv. Calculo proporciones (Ej: Cantidades promedio, Visa/Mastercard, etc).

In [80]:
%%sql
create or replace table competencia_02_sumas as
select *
    , division_segura(m_plazofijo_total, cplazo_fijo) as m_promedio_plazofijo_total
    , division_segura(m_inversion1_total, cinversion1) as m_promedio_inversion_total
    , division_segura(mcaja_ahorro, ccaja_ahorro) as m_promedio_caja_ahorro
    , division_segura(mtarjeta_visa_consumo, ctarjeta_visa_transacciones) as m_promedio_tarjeta_visa_consumo_por_transaccion
    , division_segura(mtarjeta_master_consumo, ctarjeta_master_transacciones) as m_promedio_tarjeta_master_consumo_por_transaccion
    , division_segura(mprestamos_prendarios, cprestamos_prendarios) as m_promedio_prestamos_prendarios
    , division_segura(mprestamos_hipotecarios, cprestamos_hipotecarios) as m_promedio_prestamos_hipotecarios
    , division_segura(minversion2, cinversion2) as m_promedio_inversion2
    , division_segura(mpagodeservicios, cpagodeservicios) as m_promedio_pagodeservicios
    , division_segura(mpagomiscuentas, cpagomiscuentas) as m_promedio_pagomiscuentas
    , division_segura(mcajeros_propios_descuentos, ccajeros_propios_descuentos) as m_promedio_cajeros_propios_descuentos
    , division_segura(mtarjeta_visa_descuentos, ctarjeta_visa_descuentos) as m_promedio_tarjeta_visa_descuentos
    , division_segura(mtarjeta_master_descuentos, ctarjeta_master_descuentos) as m_promedio_tarjeta_master_descuentos
    , division_segura(mcomisiones_mantenimiento, ccomisiones_mantenimiento) as m_promedio_comisiones_mantenimiento
    , division_segura(mcomisiones_otras, ccomisiones_otras) as m_promedio_comisiones_otras
    , division_segura(mforex_buy, cforex_buy) as m_promedio_forex_buy
    , division_segura(mtransferencias_recibidas, ctransferencias_recibidas) as m_promedio_transferencias_recibidas
    , division_segura(mtransferencias_emitidas, ctransferencias_emitidas) as m_promedio_transferencias_emitidas
    , division_segura(mextraccion_autoservicio, cextraccion_autoservicio) as m_promedio_extraccion_autoservicio
    , division_segura(mcheques_depositados, ccheques_depositados) as m_promedio_cheques_depositados
    , division_segura(mcheques_emitidos, ccheques_emitidos) as m_promedio_cheques_emitidos
    , division_segura(mcheques_depositados_rechazados, ccheques_depositados_rechazados) as m_promedio_cheques_depositados_rechazados
    , division_segura(mcheques_emitidos_rechazados, ccheques_emitidos_rechazados) as m_promedio_cheques_emitidos_rechazados
    , division_segura(matm, catm_trx) as m_promedio_atm
    , division_segura(matm_other, catm_trx_other) as m_promedio_atm_other
    , division_segura(Master_msaldototal,Master_mfinanciacion_limite) as proporcion_financiacion_master_cubierto
    , division_segura(Master_msaldototal,Master_mlimitecompra) as proporcion_limite_master_cubierto
    , division_segura(Visa_msaldototal,Visa_mfinanciacion_limite) as proporcion_financiacion_visa_cubierto
    , division_segura(Visa_msaldototal,Visa_mlimitecompra) as proporcion_limite_visa_cubierto
    , division_segura(tc_saldototal_total,tc_financiacionlimite_total) as proporcion_financiacion_total_cubierto
    , division_segura(tc_saldototal_total,tc_limitecompra_total) as proporcion_limite_total_cubierto
    , division_segura(tc_saldopesos_total,tc_saldototal_total) as tc_proporcion_saldo_pesos
    , division_segura(tc_saldodolares_total,tc_saldototal_total) as tc_proporcion_saldo_dolares
    , division_segura(tc_consumopesos_total,tc_consumototal_total) as tc_proporcion_consumo_pesos
    , division_segura(tc_consumototal_total,tc_limitecompra_total) as tc_proporcion_consumo_total_limite_total_cubierto
    , division_segura(tc_pagadopesos_total,tc_pagado_total) as tc_proporcion_pago_pesos
    , division_segura(tc_pagadodolares_total,tc_pagado_total) as tc_proporcion_pago_dolares
    , division_segura(tc_adelantopesos_total,tc_adelanto_total) as tc_proporcion_adelanto_pesos
    , division_segura(tc_adelantodolares_total,tc_adelanto_total) as tc_proporcion_adelanto_dolares
from competencia_02_sumas

,Success


## C. Creación de variables intermensuales (Data Drifting).

#### 1. Campos a trabajar/crear.

In [81]:
campos_iniciales = [
    "mrentabilidad","mrentabilidad_annual","mcomisiones","mactivos_margen","mpasivos_margen","cproductos","mcuenta_corriente",
    "ccaja_ahorro","mcaja_ahorro","mcaja_ahorro_adicional","mcaja_ahorro_dolares","mcuentas_saldo","ctarjeta_debito","ctarjeta_debito_transacciones",
    "mautoservicio","ctarjeta_visa_transacciones","mtarjeta_visa_consumo","ctarjeta_master_transacciones","mtarjeta_master_consumo",
    "cprestamos_prendarios","mprestamos_prendarios","cprestamos_hipotecarios","mprestamos_hipotecarios",
    "cplazo_fijo","mplazo_fijo_dolares","mplazo_fijo_pesos","cinversion1","minversion1_pesos","minversion1_dolares","cinversion2",
    "minversion2",
    "mpayroll",
    "mpayroll2","ccuenta_debitos_automaticos","mcuenta_debitos_automaticos","ctarjeta_visa_debitos_automaticos",
    "mttarjeta_master_debitos_automaticos","cpagodeservicios","mpagodeservicios","cpagomiscuentas","mpagomiscuentas","ccajeros_propios_descuentos","mcajeros_propios_descuentos",
    "ctarjeta_visa_descuentos","mtarjeta_visa_descuentos","ctarjeta_master_descuentos","mtarjeta_master_descuentos","ccomisiones_mantenimiento",
    "mcomisiones_mantenimiento","ccomisiones_otras","mcomisiones_otras","mforex_buy",
    "ctransferencias_recibidas","mtransferencias_recibidas","ctransferencias_emitidas","mtransferencias_emitidas",
    "cextraccion_autoservicio","mextraccion_autoservicio","ccheques_depositados","mcheques_depositados","ccheques_emitidos","mcheques_emitidos",
    "ccheques_depositados_rechazados","mcheques_depositados_rechazados","ccheques_emitidos_rechazados","mcheques_emitidos_rechazados",
    "chomebanking_transacciones","ccajas_consultas","ccajas_depositos",
    "ccajas_otras","catm_trx","matm","catm_trx_other","matm_other","ctrx_quarter","Master_msaldototal",
    "Master_msaldopesos","Master_msaldodolares","Master_mconsumospesos","Master_mconsumosdolares","Master_mlimitecompra","Master_madelantopesos",
    "Master_madelantodolares","Master_mpagado","Master_mpagospesos","Master_mpagosdolares","Master_mconsumototal","Master_cconsumos",
    "Master_cadelantosefectivo","Visa_msaldototal","Visa_msaldopesos","Visa_msaldodolares","Visa_mconsumospesos","Visa_mlimitecompra",
    "Visa_madelantopesos","Visa_madelantodolares","Visa_mpagospesos","Visa_mpagosdolares","Visa_mconsumototal","Visa_cconsumos","Visa_cadelantosefectivo","cmobile_app_trx",
    "Visa_mpagado"
]

campos_adicionales = [
    "tc_consumo_total","tc_financiacionlimite_total","tc_saldopesos_total","tc_saldodolares_total","tc_consumopesos_total",
    "tc_limitecompra_total","tc_adelantopesos_total","tc_adelantodolares_total","tc_adelanto_total",
    "tc_pagadopesos_total","tc_pagadodolares_total","tc_saldototal_total","tc_consumototal_total","tc_cconsumos_total",
    "tc_morosidad_total",
    "tc_pagado_total",
    "m_promedio_plazofijo_total","m_promedio_inversion_total","m_promedio_caja_ahorro","m_promedio_tarjeta_visa_consumo_por_transaccion",
    "m_promedio_tarjeta_master_consumo_por_transaccion","m_promedio_prestamos_prendarios","m_promedio_prestamos_hipotecarios","m_promedio_inversion2",
    "m_promedio_pagodeservicios","m_promedio_pagomiscuentas","m_promedio_cajeros_propios_descuentos","m_promedio_tarjeta_visa_descuentos",
    "m_promedio_tarjeta_master_descuentos","m_promedio_comisiones_mantenimiento","m_promedio_comisiones_otras","m_promedio_forex_buy",
    "m_promedio_transferencias_recibidas","m_promedio_transferencias_emitidas","m_promedio_extraccion_autoservicio",
    "m_promedio_cheques_depositados","m_promedio_cheques_emitidos","m_promedio_cheques_depositados_rechazados","m_promedio_cheques_emitidos_rechazados",
    "m_promedio_atm","m_promedio_atm_other","proporcion_financiacion_master_cubierto","proporcion_limite_master_cubierto","proporcion_financiacion_visa_cubierto",
    "proporcion_limite_visa_cubierto","proporcion_financiacion_total_cubierto","proporcion_limite_total_cubierto","tc_proporcion_consumo_total_limite_total_cubierto",
    "tc_proporcion_saldo_pesos","tc_proporcion_saldo_dolares","tc_proporcion_consumo_pesos","tc_proporcion_adelanto_pesos",
    "tc_proporcion_adelanto_dolares",
    "tc_proporcion_pago_pesos",
    "tc_proporcion_pago_dolares"
]

campos = campos_iniciales + campos_adicionales


#### 2. Referencia entrenar información con ciertos meses, y df -ventana meses.

In [82]:
#a. Pendiente de la regresión lineal.
#i. Creo los features para obtener la pendiente de la regresión lineal.
nuevos_features =""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_{ventana} as {campo}_slope_{ventana}"

In [83]:
#ii. Ejecuto.

In [84]:
%%sql
create or replace table competencia_02_sumas_drifting as
select *
  {{nuevos_features}}
from competencia_02_sumas
window ventana_{{ventana}} as (partition by numero_de_cliente order by foto_mes rows between {{ventana}} preceding and current row)


,Success


In [85]:
#b. Calculo valor actual vs acumulado historico y vs promedio histórico.
#i. Creo los features necesarios para dicho fin.
nuevos_features = ""
for campo in campos:
    nuevos_features += f'''
                         , division_segura({campo}, avg(COALESCE({campo}, 0)) over ventana_hist) as ratio_actual_{campo}
                         , sum(COALESCE({campo}, 0)) over ventana_hist as sumcum_{campo}
                         , avg(COALESCE({campo}, 0)) over ventana_hist as avg_historico_{campo}
                         , lag({campo}, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_{campo}
                          , lag({campo}, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_{campo}
                          , lag({campo}, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_{campo}
                          , {campo} - lag_1_{campo} as delta_1_{campo}
                          , {campo} - lag_2_{campo} as delta_2_{campo}
                          , {campo} - lag_3_{campo} as delta_3_{campo}
                         '''

In [56]:
#ii. Ejecuto.

In [86]:
%%sql
create or replace table competencia_02_sumas_drifting as
select *
  {{nuevos_features}}
from competencia_02_sumas_drifting
window ventana_hist as (partition by numero_de_cliente order by foto_mes rows between {{ventana}} preceding and current row);

,Success


In [102]:
%%sql
select delta_3_mcaja_ahorro from competencia_02_sumas_drifting limit 100 offset 3000000

,delta_3_mcaja_ahorro
0,12013.562635
1,2204.743449
2,-189.452947
3,-14975.859156
4,2112.489589
...,...
95,-55914.824451
96,-120371.468993
97,-112760.167398
98,-59423.938203


In [114]:
#c. Exportación.

In [103]:
# Crear el array con las rutas completas de los archivos .parquet
files = [datasets_path + f"{mes}_{ventana}_real.parquet" for mes in mes_all]
for i in range(0, len(mes_all)):
    # Ejecutar el comando COPY para el mes actual
    mes = mes_all[i]
    file = files[i]
    consulta = f"""
    COPY (SELECT * FROM competencia_02_sumas_drifting 
    WHERE foto_mes = {mes}) z
    TO '{file}' (FORMAT PARQUET);
    """
    # Ejecutar la consulta generada
    %sql {{consulta}}


RuntimeError: Query interrupted

In [32]:
#%%sql
#COPY competencia_02_sumas_drifting TO '{{dataset_output}}' (FORMAT PARQUET);

,Success
